In [19]:
import os
import pandas as pd


In [21]:
from context import record_main as recmain
import treatrec as treat
from treatrec import ekg_to_hr

In [3]:
def load(tfile = 'M2020_2_4-9_49_5.csv',
         wfile = 'M2020_2_4-9_49_5Wave.csv',
         dir_loc = '~/enva/clinique/recordings/anesthRecords/onPanelPcRecorded'):
    
    #files:
    if os.path.isfile(tfile):
        trend_filename = tfile
    else:
        trend_filename = os.path.join(dir_loc, tfile)
        
    if os.path.isfile(wfile):
        wave_filename = wfile
    else:
        wave_filename = os.path.join(dir_loc, wfile)  
        
    #trends
    monitorTrend = recmain.MonitorTrend(trend_filename)
    params = recmain.build_param_dico(file=tfile, source='monitorTrend')
    #waves
    monitorWave = recmain.MonitorWave(wave_filename)
    params = recmain.build_param_dico(file=wfile, source='monitorWave')
    params['fs'] = float(monitorWave.header['Data Rate (ms)'])*60/1000
    params['kind'] = 'as3'
    monitorWave.param = params
    #remove unnecessary waves
    for item in ['wflow', 'wawp', 'wvp']:
        del monitorWave.data[item]
    return monitorTrend, monitorWave


In [5]:
monitorTrend, monitorWave = load(tfile='../data/recordingTest/trend.csv',
                                 wfile='../data/recordingTest/wave.csv')

loading  wave.csv


In [23]:
params = monitorWave.param
print(params)

{'item': 1, 'xmin': None, 'xmax': None, 'ymin': 0, 'ymax': None, 'path': '/home/yzerlaut/work/anesthPlot/data', 'unit': 'min', 'save': False, 'memo': False, 'file': '../data/recordingTest/wave.csv', 'source': 'monitorWave', 'fs': 300.0, 'kind': 'as3'}


In [22]:
#NB data = monitorWave.data
# build a dataframe to work with (waves)
ekg_df = pd.DataFrame(monitorWave.data.wekg)*(-1)

#low pass filtering
ekg_df['wekg_lowpass'] = recmain.wf.fix_baseline_wander(ekg_df.wekg,
                                                monitorWave.param['fs'])
# beats locations (beat based dataFrame)
beat_df = treat.ekg_to_hr.detect_beats(ekg_df.wekg_lowpass, params)

source = Python port of BaselineWanderRemovalMedian.m from ECG-kit
Alex Page, alex.page@rochester.edu
https://bitbucket.org/atpage/baselinewanderremoval/src/master/


In [25]:
beat_df

,pLoc,yLoc,prominences,left_bases,right_bases
0,120086,-1.556,2.314,119427,125020
1,121116,-0.973,1.647,120090,122216
2,121675,-0.732,1.126,121360,121780
3,122183,-0.892,1.370,121360,122216
4,122770,-0.856,1.335,122216,122795
5,123324,-0.965,1.357,122216,124052
6,123882,-0.956,1.171,123749,124052
7,124443,-1.417,2.284,120090,124581
8,124991,-1.453,2.368,120090,125020
9,125542,-1.417,1.946,125020,125864
